# Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df = vacation_df.dropna()
vacation_df

,City_ID,City,Max Temp,Lat,Lng,Country,Current Description,Hotel Name
0,8,Avarua,77.05,-21.2078,-159.7750,CK,light rain,Paradise Inn
1,11,Puerto Ayora,75.69,-0.7393,-90.3518,EC,overcast clouds,Finch Bay Galapagos Hotel
2,13,Gizo,83.75,-8.1030,156.8419,SB,light rain,Imagination Island
3,19,Hobart,76.12,-42.8794,147.3294,AU,clear sky,St Ives Hobart Accommodation Tasmania
4,20,Key West,75.20,24.5557,-81.7826,US,scattered clouds,Fairfield Inn & Suites by Marriott Key West
...,...,...,...,...,...,...,...,...
124,550,San-Pedro,78.13,4.7485,-6.6363,CI,light rain,Hotel Maria
125,566,Kirakira,83.62,-10.4544,161.9205,SB,overcast clouds,SanBiz Lodge
126,568,Manggar,81.90,-2.8833,108.2667,ID,broken clouds,Guest Hotel
127,569,Tura,75.65,25.5198,90.2201,IN,clear sky,Hotel RIKMAN Continental


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}<dd>
<dt>City</dt><dd>{City}<dd>
<dt>Country</dt><dd>{Country}<dd>
<dt>Current Description</dt><dd>{Max Temp} °F</dd>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
vacation_start = vacation_df.loc[vacation_df["City"] == "Souillac"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Souillac"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Mahebourg" ]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Bambous Virieux"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Grand Gaube"] 

In [6]:
vacation_start

,City_ID,City,Max Temp,Lat,Lng,Country,Current Description,Hotel Name
42,172,Souillac,84.36,-20.5167,57.5167,MU,light rain,Shanti Maurice Resort & Spa


In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = pd.DataFrame({"Lat": vacation_start["Lat"], "Lng": vacation_start["Lng"]}).to_numpy()
end = pd.DataFrame({"Lat": vacation_end["Lat"], "Lng": vacation_end["Lng"]}).to_numpy()
stop1 = pd.DataFrame({"Lat": vacation_stop1["Lat"], "Lng": vacation_stop1["Lng"]}).to_numpy()
stop2 = pd.DataFrame({"Lat": vacation_stop2["Lat"], "Lng": vacation_stop2["Lng"]}).to_numpy()
stop3 = pd.DataFrame({"Lat": vacation_stop3["Lat"], "Lng": vacation_stop3["Lng"]}).to_numpy()

In [8]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
waypoints = [stop1[0], stop2[0], stop3[0]]
fig = gmaps.figure()

Souillac = gmaps.directions_layer(start[0], end[0], waypoints = waypoints, 
                                      travel_mode = "DRIVING")
fig.add_layer(Souillac)


fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1,
                         vacation_stop2, vacation_stop3,
                         vacation_end],ignore_index=True)
itinerary_df

,City_ID,City,Max Temp,Lat,Lng,Country,Current Description,Hotel Name
0,172,Souillac,84.36,-20.5167,57.5167,MU,light rain,Shanti Maurice Resort & Spa
1,413,Mahebourg,84.25,-20.4081,57.7000,MU,broken clouds,Shandrani Beachcomber Resort & Spa
2,104,Bambous Virieux,84.22,-20.3428,57.7575,MU,broken clouds,Casa Tia Villa
3,35,Grand Gaube,84.38,-20.0064,57.6608,MU,overcast clouds,Veranda Paul et Virginie Hotel & Spa
4,172,Souillac,84.36,-20.5167,57.5167,MU,light rain,Shanti Maurice Resort & Spa


In [14]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))